In [121]:
from dataclasses import dataclass
from typing import *
from abc import ABC, abstractmethod, abstractclassmethod


@dataclass
class TextElement():
    """
    Just a simple placeholder for text with a possibility to be extended (ex. with metadata)
    """

    string: str
    Operation_type: str
    position: Optional[int] = None



class Stack(ABC):
    """
    An abstract class for all the stack operations
    
    (Could have inhererited from list, but it has massive functional overhead in terms of methods
    from which we basically need a few")
    """
    
    def __init__(self) -> None:

        self.__stack = []

        return

    def __iter__(self) -> Iterable:
        '''
        Return stack as iterable
        '''
        for item in self.__stack:
            yield item
        return

    def __len__(self):
        """
        Get length of the current stack
        """        
        return len(self.__stack)

    def __repr__(self) -> str:
        return f"{self.__stack}"

    def __getitem__(self, index: int) -> Any:
        return self.__stack[index]

    def size(self) -> int:
        return len(self.__stack)


    def pop(self, index: int) -> List:
        '''
        Take top item (self-explanatory)
        '''
        if len(self.__stack) == 0:
            raise IndexError("Can't pop from empty list")

        return self.__stack.pop()

    
    def push(self, element: Any, position: Optional[int] = None) -> None:
        '''
        Add element to stack
        '''

        if not position is None:
            if type(position) != "int":
                raise ValueError("Position must be of type integer")

            if position >= len(self.__stack):
                raise ValueError("Position index out of range")

            return self.__stack.insert(position, element)

        return self.__stack.append(element)
        
            
    
    def clear(self) -> None:
        '''
        Empty the stack
        '''
        return self.__stack.clear()


class Text(Stack):
    """
    Represent text
    """

    def __str__(self) -> str:
        return " ".join([x for x in self.__stack])

    def __repl__(self) -> str:
        return " ".join([x for x in self.__stack])

    def show(self) -> str:

        return self.__str__()



    

class Operation(ABC):
    '''
    Operation on text
    '''
    @abstractclassmethod
    def __call__(self, string: str, position: Optional[int], length: Optional[int]) -> TextElement:
        pass


class Add(Operation):

    def __call__(self, string, position: Optional[int], length: Optional[int] = None):

        return super()

        

In [111]:
stack1 = Stack()


In [112]:
stack2 = Stack()

In [117]:
stack1.push(1)

stack2.push(3)

In [118]:
stack = stack1 + stack2

(stack, stack1, stack2)

(None, [1, 1, 1, 1, 1], [3, 3, 3, 3, 3])

In [45]:
Change

[12, 2, 3, [2], 4, 5, 6, [2], [2], [3, 4]]

In [35]:
Change.clear()

Change.pop()

IndexError: pop from empty list

In [93]:
dir(list)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']